In [1]:
import pandas as pd

In [2]:
df = pd.read_csv ("../data/train/vaccination/fr/vaccination_hist_data.csv", sep =";")
df['departement'] = df['departement'].replace({'2A':'201','2B':'202'}).astype(int)
df = df[df['departement']<203]
df["date_debut_semaine"]=pd.to_datetime(df["date_debut_semaine"], dayfirst = True) 

In [3]:
df2 = pd.read_csv("../data/train/all_data_merged/fr/Enriched_Covid_history_data.csv", sep = ",")
df2["time"]=pd.to_datetime(df2["time"])

In [4]:
dfvac1 = df[df["rang_vaccinal"]==1].reset_index()
dfvac2 = df[df["rang_vaccinal"]==2].reset_index()

In [5]:
referencedate1 = dfvac1['date_debut_semaine'].min()
referencedate2 = dfvac2['date_debut_semaine'].min()
#print('vac1 :', referencedate1, '\n', 'vac2 :', referencedate2)

In [6]:
cum1 = dfvac1.groupby(['departement', 'date_debut_semaine']).sum().groupby(level=0).cumsum().sort_values("date_debut_semaine").reset_index().drop(columns = "index")
cum2 = dfvac2.groupby(['departement', 'date_debut_semaine']).sum().groupby(level=0).cumsum().sort_values("date_debut_semaine").reset_index().drop(columns = "index")

In [7]:
def create_possibilities(row):
    return pd.date_range(start=row['date_debut_semaine'], periods=7).tolist()


cum1['7_days'] = cum1.apply(create_possibilities, axis=1)
cum2['7_days'] = cum2.apply(create_possibilities, axis=1)

In [8]:
def check_variant(v_row, date):
    if date in v_row['7_days']:
#        print(v_row['nb'])
        return v_row['nb']


In [9]:
def enriched_variant(row):
    date = row['time']
    depnum = row['numero']
    if date < referencedate1:
        row['vac1nb'], row['vac2nb'] = 0, 0
    else:
        cum1_dep = cum1[cum1['departement'] == depnum]
        res1 = cum1_dep.apply(check_variant, date=date, axis=1)
        first1 = [el for el in res1 if el == el][0]  #get the first non null element of res 
        
        cum2_dep = cum2[cum2['departement'] == depnum]
        res2 = cum2_dep.apply(check_variant, date=date, axis=1)
#        print(res2)
        first2 = next((el for el in res2 if el==el),None) #get the first non null element of res
#        print(date)
        if first2 is None:
            first2 = 0
#        print('both :',first1, first2)
        row['vac1nb']= first1
        row['vac2nb'] = first2
    return None

In [10]:
df2.apply(enriched_variant, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
30490    None
30491    None
30492    None
30493    None
30494    None
Length: 30495, dtype: object

In [ ]:
df2

In [11]:
df3 = pd.read_csv("../data/train/all_data_merged/fr/Enriched_Covid_history_data.csv", sep = ",")
df3["time"]=pd.to_datetime(df3["time"])
test = df3[['time', 'vac1nb', 'vac2nb']] == df2[['time', 'vac1nb', 'vac2nb']]

In [13]:
test['vac2nb'].unique()

array([ True])

In [ ]:
essai = pd.DataFrame([[1, 2, 3],[4, 5, 6]], columns=['a', 'b', 'c'])
essai

In [ ]:
def my_func(row):
    row['b']=10
    return None

In [ ]:
essai.apply(my_func, axis=1)

In [ ]:
essai